In [ ]:
####################################################################################################
# CR.Screening.AOD_WV_Precip_Tau.ipynb
# Karl Lapo September/2015
####################################################################################################
# Aggregating data according to AOD, WV, Precip, and Tau. Figures for CalRad Paper
####################################################################################################

# must insert this statement to render the plots within the notebook
# this is specific to the ipython notebook
%matplotlib inline

## Import statements
# netcdf/numpy/xray/stats
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import xray
import kray #### Custom functions for manipulating xray structures
from scipy import interpolate as interp
from scipy.stats.stats import pearsonr
from scipy import stats

# OS interaction
import sys, pickle, os

# import subplots function for plotting
import seaborn as sns
import matplotlib
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap
import kgraph

# Solargeometry
import solargeo

## Directory listing
dir_data = '/Users/karllapo/gdrive/SnowHydrology/proj/CloudClimatology/data'
dir_print = '/Users/karllapo/gdrive/SnowHydrology/proj/CloudClimatology/Graphics'

# Directory for basemap pickle files
dir_bmap = '/Users/karllapo/gdrive/SnowHydrology/proj/CloudClimatology/data/basemap'

In [ ]:
#### Load processed data (CR.xrayDS_format)

In [ ]:
#### Anomaly vs bias, no rain
# groups_for_plot = ['north cv','south cv']
groups_for_plot = groups
colsche = kgraph.SRON(len(np.atleast_1d(groups_for_plot)))
f,axes = plt.subplots(2,2,figsize=(10,10),sharex='col',sharey='row')
axes = axes.flatten()

# Rain screening
pthresh = 1 # More than 1mm group average
grobs_diff_pthresh = grobs_diff.where(precip < pthresh)
anomp = anom.where(precip < pthresh)

## Scatter plots
for m,pr in enumerate(pr_names):
    ax = axes[m]
    for n,gr in enumerate(groups_for_plot):
        # axis lines
        ax.plot(np.array((-1,1)),np.array((0,0)),'k--')
        ax.plot(np.array((0,0)),np.array((-1000,1000)),'k--')
        
        # Daily anomaly and bias
        ax.plot(anomp.anom_mean.loc[{'Grouping':gr}],\
                grobs_diff_pthresh[pr].loc[{'Grouping':gr}],\
                '.',color=colsche[n],label=gr)
    
        # Linear fit
        x = anomp.anom_mean.loc[{'Grouping':gr}]
        y = grobs_diff_pthresh[pr].loc[{'Grouping':gr}]
        xplot = x[np.logical_and(~np.isnan(x),~np.isnan(y))]
        yplot = y[np.logical_and(~np.isnan(x),~np.isnan(y))]
       
        m0,b0,r,p,std_err = stats.linregress(xplot,yplot)
        line = m0*xi + b0
        ax.plot(xi,line,color=colsche[n])
#         print(pr+' @ '+gr+': ')
#         print('\t r2= '+str(r**2))
#         print('\t m0= '+str(m0))
#         print('\t b0= '+str(b0))
        
        
    ## Formatting
    if np.mod(m,2) == 0:
        ax.set_ylabel('Bias Wm$^{-2}$')
    if m > 1:
        ax.set_xlabel('$\Delta$ $\\tau$')
    ax.set_title(pr)
    ax.set_ylim(-200,200)
    ax.set_xlim(-.6,.6)

ax.legend()
plt.tight_layout()

# Output
os.chdir(dir_print)
fname = 'Tau_Bias.DailyScatter.NoRain.png'
f.savefig(fname)

In [ ]:
#### Anomaly vs bias, RAIN > 1mm
# groups_for_plot = ['north cv','south cv']
groups_for_plot = groups
colsche = kgraph.SRON(len(np.atleast_1d(groups_for_plot)))
f,axes = plt.subplots(2,2,figsize=(10,10),sharex='col',sharey='row')
axes = axes.flatten()

# Rain screening
pthresh = 1 # More than 1mm group average
grobs_diff_pthresh = grobs_diff.where(precip >= pthresh)
anomp = anom.where(precip >= pthresh)
xi = np.arange(-1000,1000,50)

## Scatter plots
for m,pr in enumerate(pr_names):
    ax = axes[m]
    for n,gr in enumerate(groups_for_plot):
        # axis lines
        ax.plot(np.array((-1,1)),np.array((0,0)),'k--')
        ax.plot(np.array((0,0)),np.array((-1000,1000)),'k--')
        
        # Daily anomaly and bias
        ax.plot(anomp.anom_mean.loc[{'Grouping':gr}],\
                grobs_diff_pthresh[pr].loc[{'Grouping':gr}],\
                '.',color=colsche[n],label=gr)
    
        # Linear fit
        x = anomp.anom_mean.loc[{'Grouping':gr}]
        y = grobs_diff_pthresh[pr].loc[{'Grouping':gr}]
        xplot = x[np.logical_and(~np.isnan(x),~np.isnan(y))]
        yplot = y[np.logical_and(~np.isnan(x),~np.isnan(y))]
       
        m0,b0,r,p,std_err = stats.linregress(xplot,yplot)
        line = m0*xi + b0
        ax.plot(xi,line,color=colsche[n])
#         print(pr+' @ '+gr+': ')
#         print('\t r2= '+str(r**2))
#         print('\t m0= '+str(m0))
#         print('\t b0= '+str(b0))
        
    ## Formatting
    if np.mod(m,2) == 0:
        ax.set_ylabel('Bias Wm$^{-2}$')
    if m > 1:
        ax.set_xlabel('$\Delta$ $\\tau$')
    ax.set_title(pr)
    ax.set_ylim(-200,200)
    ax.set_xlim(-.6,.6)

ax.legend()
plt.tight_layout()

# Output
os.chdir(dir_print)
fname = 'Tau_Bias.DailyScatter.Rain.png'
f.savefig(fname)

In [ ]:
#### Anomaly vs bias, AOD screening
groups_for_plot = groups
colsche = kgraph.SRON(len(np.atleast_1d(groups_for_plot)))
f,axes = plt.subplots(4,2,figsize=(8,12),sharex='col',sharey='row')

################
# Smoggy days ##
################
# Rain screening
aod_thresh = .2 # Aerosol Optical Depth > .2
# aerosol screening
grobs_diff_aodthresh = grobs_diff.where(grobs_diff.aod >= aod_thresh)
anom_aodthres = anom.where(anom.aod >= aod_thresh)

## Scatter plots
for m,pr in enumerate(pr_names):
    ax = axes[m,0]
    for n,gr in enumerate(groups_for_plot):
        # axis lines
        ax.plot(np.array((-1,1)),np.array((0,0)),'k--')
        ax.plot(np.array((0,0)),np.array((-1000,1000)),'k--')
        
        # Daily anomaly and bias
        ax.plot(anom_aodthres.anom_mean.loc[{'Grouping':gr}],\
                grobs_diff_aodthresh[pr].loc[{'Grouping':gr}],\
                '.',color=colsche[n],label=gr)
    
        # Linear fit
        x = anom_aodthres.anom_mean.loc[{'Grouping':gr}]
        y = grobs_diff_aodthresh[pr].loc[{'Grouping':gr}]
        xplot = x[np.logical_and(~np.isnan(x),~np.isnan(y))]
        yplot = y[np.logical_and(~np.isnan(x),~np.isnan(y))]
       
        m0,b0,r,p,std_err = stats.linregress(xplot,yplot)
        line = m0*xi + b0
        ax.plot(xi,line,color=colsche[n])
              
    ## Formatting
    ax.set_ylabel('Bias Wm$^{-2}$')
    if m == len(pr_names):
        ax.set_xlabel('$\Delta$ $\\tau$')
    ax.set_title(pr+' (smog)')
    ax.set_ylim(-200,200)
    ax.set_xlim(-.6,.6)
    
###################
# Smog free days ##
###################
# aerosol screening
grobs_diff_aodthresh = grobs_diff.where(grobs_diff.aod < aod_thresh)
anom_aodthres = anom.where(anom.aod < aod_thresh)

## Scatter plots
for m,pr in enumerate(pr_names):
    ax = axes[m,1]
    for n,gr in enumerate(groups_for_plot):
        # axis lines
        ax.plot(np.array((-1,1)),np.array((0,0)),'k--')
        ax.plot(np.array((0,0)),np.array((-1000,1000)),'k--')
        
        # Daily anomaly and bias
        ax.plot(anom_aodthres.anom_mean.loc[{'Grouping':gr}],\
                grobs_diff_aodthresh[pr].loc[{'Grouping':gr}],\
                '.',color=colsche[n],label=gr)
    
        # Linear fit
        x = anom_aodthres.anom_mean.loc[{'Grouping':gr}]
        y = grobs_diff_aodthresh[pr].loc[{'Grouping':gr}]
        xplot = x[np.logical_and(~np.isnan(x),~np.isnan(y))]
        yplot = y[np.logical_and(~np.isnan(x),~np.isnan(y))]
       
        m0,b0,r,p,std_err = stats.linregress(xplot,yplot)
        line = m0*xi + b0
        ax.plot(xi,line,color=colsche[n])

    ## Formatting
    if m == 0:
        ax.legend(loc='best',frameon=True)
    if m == len(pr_names):
        ax.set_xlabel('$\Delta$ $\\tau$')
    ax.set_title(pr+' (no smog)')
    ax.set_ylim(-200,200)
    ax.set_xlim(-.6,.6)

plt.tight_layout()

# Output
os.chdir(dir_print)
fname = 'Tau_Bias.DailyScatter.AOD.png'
f.savefig(fname)

In [ ]:
#### Anomaly vs bias, rain screening
groups_for_plot = groups
colsche = kgraph.SRON(3)

# Rain screening
pthresh = 1 # More than 1mm group average
grobs_diff_p = grobs_diff.where(precip >= pthresh)
anomp = anom.where(precip >= pthresh)
grobs_diff_nop = grobs_diff.where(precip < pthresh)
anom_nop = anom.where(precip < pthresh)


## Scatter plots
for m,pr in enumerate(pr_names):
    f,axes = plt.subplots(3,2,figsize=(8,12),sharex='col',sharey='row')
    axes = axes.flatten()

    for n,gr in enumerate(groups_for_plot):
        # axis lines
        ax = axes[n]
        ax.plot(np.array((-1,1)),np.array((0,0)),'k--')
        ax.plot(np.array((0,0)),np.array((-1000,1000)),'k--')
        
        ### Daily anomaly and bias - rain ###
        x = anom_nop.anom_mean.loc[{'Grouping':gr}]
        y = grobs_diff_nop[pr].loc[{'Grouping':gr}]
        
        # Dialy scatter points
        ax.plot(x,y,'o',color=colsche[-1],label='no rain')
    
        # Linear fit
        xplot = x[np.logical_and(~np.isnan(x),~np.isnan(y))]
        yplot = y[np.logical_and(~np.isnan(x),~np.isnan(y))]
        m0,b0,r,p,std_err = stats.linregress(xplot,yplot)
        line = m0*xi + b0
        ax.plot(xi,line,color=colsche[-1])
        
        ### Daily anomaly and bias - rain ###
        x = anomp.anom_mean.loc[{'Grouping':gr}]
        y = grobs_diff_p[pr].loc[{'Grouping':gr}]
        
        # Dialy scatter points
        ax.plot(x,y,'.',color=colsche[0],label='rain')

        # Linear fit
        xplot = x[np.logical_and(~np.isnan(x),~np.isnan(y))]
        yplot = y[np.logical_and(~np.isnan(x),~np.isnan(y))]
        m0,b0,r,p,std_err = stats.linregress(xplot,yplot)
        line = m0*xi + b0
        ax.plot(xi,line,color=colsche[0])
  
        ## Formatting
        ax.set_xlabel('$\Delta$ $\\tau$')
        ax.set_ylim(-200,200)
        ax.set_xlim(-.6,.6)
        ax.set_title(pr+' @ '+gr)
        ax.set_ylabel('Bias Wm$^{-2}$')
        if ax == axes.flatten()[0]:
            handles, labels = ax.get_legend_handles_labels()
            axes[-1].axis('off')
            axes[-1].legend(handles, labels,loc='center left')
    
    plt.tight_layout()

    # Output
    os.chdir(dir_print)
    fname = 'Tau_Bias.DailyScatter.Option3.'+pr+'.png'
    f.savefig(fname)
    plt.close(f)

In [ ]:
#### Anomaly vs bias, no rain (option 1)
groups_for_plot = ['north cv','south cv']
# groups_for_plot = groups
colsche = kgraph.SRON(len(np.atleast_1d(groups_for_plot)))
f,axes = plt.subplots(4,2,figsize=(8,12),sharex='col',sharey='row')

# screening
aod_thresh = .2 # Aerosol Optical Depth > .2
pthresh = 1 # Group precip > 1mm/day

## Scatter plots
for m,pr in enumerate(pr_names):
    ax = axes[m,0]
    for n,gr in enumerate(groups_for_plot):
        anom_thresh = anom.where(np.logical_or(anom.aod.values >= aod_thresh,\
                                            anom.loc[{'Grouping':gr}].precip.values >= pthresh))
        grobs_diff_thresh = grobs_diff.where(np.logical_or(grobs_diff.aod.values >= aod_thresh,\
                                            grobs_diff.loc[{'Grouping':gr}].precip.values >= pthresh))
        
        # axis lines
        ax.plot(np.array((-1,1)),np.array((0,0)),'k--')
        ax.plot(np.array((0,0)),np.array((-1000,1000)),'k--')
        
        # Daily anomaly and bias
        ax.plot(anom_thresh.anom_mean.loc[{'Grouping':gr}],\
                grobs_diff_thresh[pr].loc[{'Grouping':gr}],\
                '.',color=colsche[n],label=gr)
    
        # Linear fit
        x = anom_thresh.anom_mean.loc[{'Grouping':gr}]
        y = grobs_diff_thresh[pr].loc[{'Grouping':gr}]
        xplot = x[np.logical_and(~np.isnan(x),~np.isnan(y))]
        yplot = y[np.logical_and(~np.isnan(x),~np.isnan(y))]
        
#         m0,b0,r,p,std_err = stats.linregress(xplot,yplot)
#         line = m0*xi + b0
#         ax.plot(xi,line,color=colsche[n])
              
    ## Formatting
    ax.set_ylabel('Bias Wm$^{-2}$')
    if m == len(pr_names):
        ax.set_xlabel('$\Delta$ $\\tau$')
    ax.set_title(pr+' (screen)')
    ax.set_ylim(-200,200)
    ax.set_xlim(-.6,.6)
    
## Scatter plots
for m,pr in enumerate(pr_names):
    ax = axes[m,1]
    for n,gr in enumerate(groups_for_plot):
        anom_thresh = anom.where(np.logical_and(anom.aod.values < aod_thresh,\
                                    anom.loc[{'Grouping':gr}].precip.values < pthresh))
        grobs_diff_thresh = grobs_diff.where(np.logical_and(grobs_diff.aod.values < aod_thresh,\
                                    grobs_diff.loc[{'Grouping':gr}].precip.values < pthresh))
        
        # axis lines
        ax.plot(np.array((-1,1)),np.array((0,0)),'k--')
        ax.plot(np.array((0,0)),np.array((-1000,1000)),'k--')
        
        # Daily anomaly and bias
        ax.plot(anom_thresh.anom_mean.loc[{'Grouping':gr}],\
                grobs_diff_thresh[pr].loc[{'Grouping':gr}],\
                '.',color=colsche[n],label=gr)
    
        # Linear fit
        x = anom_thresh.anom_mean.loc[{'Grouping':gr}]
        y = grobs_diff_thresh[pr].loc[{'Grouping':gr}]
        xplot = x[np.logical_and(~np.isnan(x),~np.isnan(y))]
        yplot = y[np.logical_and(~np.isnan(x),~np.isnan(y))]
        
#         m0,b0,r,p,std_err = stats.linregress(xplot,yplot)
#         line = m0*xi + b0
#         ax.plot(xi,line,color=colsche[n])

    ## Formatting
    if m == 0:
        ax.legend(loc='best',frameon=True)
    if m == len(pr_names):
        ax.set_xlabel('$\Delta$ $\\tau$')
    ax.set_title(pr+' (no screen)')
    ax.set_ylim(-200,200)
    ax.set_xlim(-.6,.6)

plt.tight_layout()

# Output
os.chdir(dir_print)
fname = 'Tau_Bias.DailyScatter.AODandPrecip.png'
f.savefig(fname)